In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_train = 100.*np.random.sample(10**4)
y_train = x_train.astype(int)

x_test = 100.*np.random.sample(10**4)
y_test = x_test.astype(int)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [3]:
EPOCHS = 100

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(1,1 ))
        self.dense1 = tf.keras.layers.Dense(32, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(64, activation = 'relu')
        self.dense3 = tf.keras.layers.Dense(128, activation = 'relu')
        self.dense4 = tf.keras.layers.Dense(256, activation = 'relu')
        self.dense5 = tf.keras.layers.Dense(100, activation = 'softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        return x

In [5]:
@tf.function
def train_step(model, in_var, labels, loss_object, optimizer, 
               train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(in_var)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [6]:
@tf.function
def test_step(model, in_var, labels, loss_object, test_loss, test_accuracy):
    predictions = model(in_var)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

In [7]:
model = MyModel()

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [9]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras. metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [10]:
for epoch in range(EPOCHS):
    for in_var, labels in train_ds:
        train_step(model, in_var, labels, loss_object, optimizer, 
               train_loss, train_accuracy)
        
    for in_var, labels in test_ds:
        test_step(model, in_var, labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch: {}, Loss: {}, Acc: {}, Test loss: {}, Test acc: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 1, Loss: 4.1799235343933105, Acc: 0.03790000081062317, Test loss: 3.507280111312866, Test acc: 0.0763000026345253
Epoch: 2, Loss: 3.0734500885009766, Acc: 0.12759999930858612, Test loss: 2.7857625484466553, Test acc: 0.1615000069141388
Epoch: 3, Loss: 2.654762029647827, Acc: 0.17970000207424164, Test loss: 2.6768105030059814, Test acc: 0.163100004196167
Epoch: 4, Loss: 2.4408762454986572, Acc: 0.20999999344348907, Test loss: 2.2329416275024414, Test acc: 0.25459998846054077
Epoch: 5, Loss: 2.2623